<a href="https://colab.research.google.com/github/inderpreetsingh01/Speech/blob/main/Model_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#problem with pyannote.audio
#hmmlearn has a new implementation which breaks the viterbi=True in Crepe

GPU 0: Tesla T4 (UUID: GPU-82d36d33-2399-09b6-c7a9-1827cf6790d2)


In [ ]:
#check that we have GPU acceleration enabled

!nvidia-smi -L

In [ ]:
!pip install torch==1.11.0 torchvision==0.12.0 torchaudio==0.11.0
!pip install pyannote.audio
# Restart run time

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 750.6 MB 10 kB/s 
     |████████████████████████████████| 21.0 MB 25.1 MB/s 
     |████████████████████████████████| 2.9 MB 81.8 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.1+cu113
    Uninstalling torchvision-0.13.1+cu113:
      Successfully uninstalled torchvision-0.13.1+cu113
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.12.1+cu113
    Uninstalling torchaudio-0.12.1+cu113:
      Successfully uninstalled torchaudio-0.12.1+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.


In [ ]:
!pip install torchtext==0.12.0
!apt-get install libsox-fmt-all libsox-dev sox
!pip install sox wget transformers  pytube pydub crepe
#restart runtime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.4 MB 16.6 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libao-common libao4 libid3tag0 libmad0 libmagic-mgc libmagic1
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-ao
  libsox-fmt-base libsox-fmt-mp3 libsox-fmt-oss libsox-fmt-pulse libsox3
Suggested packages:
  libaudio2 file
The following NEW packages will be installed:
  libao-common libao4 libid3tag0 libmad0 libmagic-mgc libmagic1
  libopencore-amrnb0 libopencore-amrw

In [ ]:
#install the necessary packages

%%capture install_log
!apt-get install libsox-fmt-all libsox-dev sox
!pip install sox pytube pydub crepe transformers torchtext==0.12.0 pyannote.audio

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libao-common libao4 libid3tag0 libmad0 libmagic-mgc libmagic1
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-ao
  libsox-fmt-base libsox-fmt-mp3 libsox-fmt-oss libsox-fmt-pulse libsox3
Suggested packages:
  libaudio2 file
The following NEW packages will be installed:
  libao-common libao4 libid3tag0 libmad0 libmagic-mgc libmagic1
  libopencore-amrnb0 libopencore-amrwb0 libsox-dev libsox-fmt-all
  libsox-fmt-alsa libsox-fmt-ao libsox-fmt-base libsox-fmt-mp3 libsox-fmt-oss
  libsox-fmt-pulse libsox3 sox
0 upgraded, 18 newly installed, 0 to remove and 20 not upgraded.
Need to get 1,267 kB of archives.
After this operation, 9,153 kB of additional disk space will be used.
Get:1 http://archi

In [ ]:
#copy all of our utility functions into the virtual environment 

!cp /content/drive/MyDrive/Omdena_Challenge/utils.py /content 

In [ ]:
#import the necessary packages

import os 
import sys
import sox
import spacy
import crepe
import librosa
import utils as u 
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pytube import YouTube
from pydub import AudioSegment
from pydub.utils import make_chunks
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from lightgbm.sklearn import LGBMRegressor
from hyperopt import tpe, hp, fmin, STATUS_OK, Trials 
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, pipeline, AutoModelForSequenceClassification, AutoTokenizer
from pyannote.audio import Pipeline, Inference

In [ ]:
#set up our file structure

u.make_directories(['input', 'tmp', 'user_data'])

In [ ]:
#create a speaker sample

u.retrieve_audio("https://www.youtube.com/watch?v=6ObqydfPGLI", "tmp")
u.convert_to_wav('tmp/Yale Professor Tony Leiserowitz Discusses American Perceptions of Climate Change.mp4', "tmp")
tfm = sox.Transformer()
tfm.trim(10, 40)
tfm.build_file('tmp/Yale Professor Tony Leiserowitz Discusses American Perceptions of Climate Change.wav', "user_data/speaker_sample.wav")
speaker_sample = "user_data/speaker_sample.wav"

In [ ]:
#retrieve and chunk additional recordings of the same speaker to simulate the activity of the user over time

u.retrieve_audio("https://www.youtube.com/watch?v=T-JVKqpvt2c&t=1996s", "tmp")
u.convert_to_wav('tmp/Dr Anthony Leiserowitzs Keynote Address - 2017 Conference.mp4', "tmp")
tfm = sox.Transformer()
tfm.trim(75, 2280)
tfm.build_file('tmp/Dr Anthony Leiserowitzs Keynote Address - 2017 Conference.wav', 'tmp/Dr_Anthony_Leiserowitzs_Keynote Address_2017_Conference_trimmed.wav')
u.chunk_audio('tmp/Dr_Anthony_Leiserowitzs_Keynote Address_2017_Conference_trimmed.wav', 'input')

In [ ]:
#import the required models 

diarization_pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization@2022.07") 
asr_processor = Wav2Vec2Processor.from_pretrained("gngpostalsrvc/w2v2-ami")
asr_model = Wav2Vec2ForCTC.from_pretrained("gngpostalsrvc/w2v2-ami")
inference = Inference("pyannote/embedding", window="whole", device="cuda")
emotion = pipeline('sentiment-analysis', model='arpanghoshal/EmoRoBERTa')
lang_model = spacy.load("en_core_web_sm")
topic_tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/tweet-topic-21-multi')
topic_model = AutoModelForSequenceClassification.from_pretrained('cardiffnlp/tweet-topic-21-multi')

Downloading:   0%|          | 0.00/598 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/318 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.88k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
output = pd.DataFrame(columns=['date', 'mutual_silence', 'overlap_duration', 'interruptions', 'total_turn_duration', 'speaking_percent', 'response_time', 'emotions', 'topic_shifts', 'words_per_minute', 'fillerword_percent', 'mean_spectral_flatness', 'spectral_flatness_std', 'mean_pitch', 'pitch_std', 'mean_volume', 'volume_std', 'is_group', 'skill', 'score'])

In [ ]:
np.random.seed(42) #set the random seed to insure reproducibility

for chunk in os.listdir('input'):

  chunk = {'name' : chunk.split(".")[0], 'audio' : f"input/{chunk}"}

  features = u.get_features(chunk, speaker_sample, diarization_pipeline, asr_processor, asr_model, emotion, topic_tokenizer, topic_model, inference, lang_model, threshold=.1)

  features['skill'] = 'communication'
  features['score'] = np.random.randint(6) 
  output = output.append(features, ignore_index=True)

91/91 [==============================] - 4s 43ms/step


https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


ValueError: ignored

In [ ]:
#clean up unneeded files

for file in os.scandir('input'):
  os.remove(file.path)

In [ ]:
#save the data to csv for later use

output.to_csv('user_data/output.csv')

In [ ]:
#examine the data 

output.head()

In [ ]:
"""generate a lgbm model based on the data and calculate the most important features (note: the MSE from the model will be relatively high 
and the important features will appear arbitrary because the independent variables remain fairly constant in this toy example)"""

model, feature_importances = u.generate_lgbm_model(output, 'communication')

In [ ]:
#visualize feature importances

fig, ax = plt.subplots(figsize=(24, 14))
sns.barplot(y='Feature', x='Importance', data=feature_importances, orient='h')
plt.title('Feature Importances for Communication')
plt.show();

In [ ]:
model.booster_.save_model('communication_model.txt')

In [ ]:
!cp /content/communication_model.txt /content/drive/MyDrive/Omdena_Challenge

In [ ]:
!cp /content/user_data/output.csv /content/drive/MyDrive/Omdena_Challenge